# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 8:56AM,261472,10,HH9285,Hush Hush,Released
1,Apr 20 2023 8:54AM,261471,10,Enova-11689,Emerginnova,Released
2,Apr 20 2023 8:54AM,261471,10,Enova-11690,Emerginnova,Released
3,Apr 20 2023 8:54AM,261470,10,EYE43048,Eye Pharma Inc,Released
4,Apr 20 2023 8:54AM,261470,10,EYE43049,Eye Pharma Inc,Released
5,Apr 20 2023 8:54AM,261470,10,EYE43050,Eye Pharma Inc,Released
6,Apr 20 2023 8:54AM,261470,10,EYE43051,Eye Pharma Inc,Released
7,Apr 20 2023 8:51AM,261468,10,CLI00535.1,"CLINUVEL, Inc.",Released
8,Apr 20 2023 8:50AM,261467,10,MSP220963,"Methapharm, Inc.",Released
9,Apr 20 2023 8:50AM,261467,10,MSP220964,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,261467,Released,4
20,261468,Released,1
21,261470,Released,4
22,261471,Released,2
23,261472,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261467,NaN,NaN,4.0
261468,NaN,NaN,1.0
261470,NaN,NaN,4.0
261471,NaN,NaN,2.0
261472,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261398,28.0,12.0,2.0
261405,0.0,0.0,2.0
261406,0.0,1.0,0.0
261410,2.0,2.0,5.0
261432,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261398,28,12,2
261405,0,0,2
261406,0,1,0
261410,2,2,5
261432,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261398,28,12,2
1,261405,0,0,2
2,261406,0,1,0
3,261410,2,2,5
4,261432,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261398,28,12,2
1,261405,,,2
2,261406,,1,
3,261410,2,2,5
4,261432,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 8:56AM,261472,10,Hush Hush
1,Apr 20 2023 8:54AM,261471,10,Emerginnova
3,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc
7,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc."
8,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc."
12,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc."
16,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc."
17,Apr 20 2023 8:31AM,261464,16,"SHL Pharma, LLC"
18,Apr 20 2023 8:19AM,261463,19,"AdvaGen Pharma, Ltd"
73,Apr 20 2023 8:07AM,261462,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 8:56AM,261472,10,Hush Hush,,,1
1,Apr 20 2023 8:54AM,261471,10,Emerginnova,,,2
2,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,,,4
3,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",,,1
4,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",,,4
5,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc.",,,4
6,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc.",,,1
7,Apr 20 2023 8:31AM,261464,16,"SHL Pharma, LLC",,,1
8,Apr 20 2023 8:19AM,261463,19,"AdvaGen Pharma, Ltd",,,55
9,Apr 20 2023 8:07AM,261462,19,"AdvaGen Pharma, Ltd",,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 8:56AM,261472,10,Hush Hush,1,,
1,Apr 20 2023 8:54AM,261471,10,Emerginnova,2,,
2,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,4,,
3,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",1,,
4,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",4,,
5,Apr 20 2023 8:46AM,261466,19,"Methapharm, Inc.",4,,
6,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc.",1,,
7,Apr 20 2023 8:31AM,261464,16,"SHL Pharma, LLC",1,,
8,Apr 20 2023 8:19AM,261463,19,"AdvaGen Pharma, Ltd",55,,
9,Apr 20 2023 8:07AM,261462,19,"AdvaGen Pharma, Ltd",12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 8:56AM,261472,10,Hush Hush,1,,
1,Apr 20 2023 8:54AM,261471,10,Emerginnova,2,,
2,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,4,,
3,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",1,,
4,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 8:56AM,261472,10,Hush Hush,1.0,NaN,NaN
1,Apr 20 2023 8:54AM,261471,10,Emerginnova,2.0,NaN,NaN
2,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,4.0,NaN,NaN
3,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",1.0,NaN,NaN
4,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 8:56AM,261472,10,Hush Hush,1.0,0.0,0.0
1,Apr 20 2023 8:54AM,261471,10,Emerginnova,2.0,0.0,0.0
2,Apr 20 2023 8:54AM,261470,10,Eye Pharma Inc,4.0,0.0,0.0
3,Apr 20 2023 8:51AM,261468,10,"CLINUVEL, Inc.",1.0,0.0,0.0
4,Apr 20 2023 8:50AM,261467,10,"Methapharm, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1568792,12.0,1.0,0.0
16,784359,4.0,0.0,0.0
19,2353004,105.0,12.0,28.0
20,522816,5.0,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1568792,12.0,1.0,0.0
1,16,784359,4.0,0.0,0.0
2,19,2353004,105.0,12.0,28.0
3,20,522816,5.0,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,12.0,1.0,0.0
1,16,4.0,0.0,0.0
2,19,105.0,12.0,28.0
3,20,5.0,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,12.0
1,16,Released,4.0
2,19,Released,105.0
3,20,Released,5.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Completed,0.0,0.0,28.0,2.0
Executing,1.0,0.0,12.0,3.0
Released,12.0,4.0,105.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Completed,0.0,0.0,28.0,2.0
1,Executing,1.0,0.0,12.0,3.0
2,Released,12.0,4.0,105.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Completed,0.0,0.0,28.0,2.0
1,Executing,1.0,0.0,12.0,3.0
2,Released,12.0,4.0,105.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()